In [ ]:
## https://www.tensorflow.org/get_started/mnist/pros

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
import tensorflow as tf
sess = tf.InteractiveSession()

## Softmax Regression Model

In [9]:
# pixel 28x28=784
# 10 output states - numbers 1-10
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [10]:
# weights and biases
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

In [11]:
# vectorized model
y = tf.matmul(x,W) + b

In [12]:
#loss function  - intrnally calls tf.nn.softmax_cross_entropy_with_logits and tf.reduce_mean 

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y,dim=1))


## Train the model

In [14]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


In [15]:
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluate the model

In [16]:
# First we'll figure out where we predicted the correct label. 
# tf.argmax is an extremely useful function which gives you the index of the highest 
# entry in a tensor along some axis. For example, tf.argmax(y,1) is the label our model 
# thinks is most likely for each input, while tf.argmax(y_,1) is the true label. 
# We can use tf.equal to check if our prediction matches the truth.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))


In [17]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9206


In [ ]:
#https://www.youtube.com/watch?v=BhpvH5DuVu8

## Build a Multilayer Convolutional Network

In [13]:
#define functions

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [16]:
# First Convolutional Layer

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [17]:
# Second Convolutional Layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)